MUSIC RECOMMENDATION SYSTEM
===========================
- ** Predict genres **
- ** Predict moods **
- ** Recommend base on cosin similarity and jaccard similarity **

In [1]:
import pickle
import pandas as pd
import numpy as np

In [2]:
# makes printing a dict look nice
import pprint
pp = pprint.PrettyPrinter(indent=4)

In [3]:
from collections import namedtuple
# Create a namedtuple called 'Song'
Song = namedtuple("Song", ["artist", "title"])
# I'm choosing to also include probability, so I'm making Mood a namedtuple as well
Mood = namedtuple("Mood", ["description", "probability"])

## Load model

In [4]:
# Load my resources
#predict_genres = pickle.load(open('predict_genres.pickle', 'rb'))
#predict_moods_lyrics  = pickle.load(open('lyrics_predict_moods.rfc.pickle', 'rb'))
#predict_moods_audio = pickle.load(open('audio_predict_moods.rfc.pickle', 'rb'))
my_database = pickle.load(open('my_database_new.pickle', 'rb'))

In [5]:
# Tools for cleaning lyrics
from string import punctuation
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from nltk.stem.snowball import SnowballStemmer

translator = str.maketrans('','', punctuation)
stemmer = SnowballStemmer('english')

def clean_text(raw_text):
    
    clean_words = []
    
    raw_text = raw_text.lower()
    raw_text = raw_text.translate(translator)
    
    for word in raw_text.split():
        
        if word not in ENGLISH_STOP_WORDS:
            
            clean_words.append(stemmer.stem(word))
    
    return " ".join(clean_words)

In [18]:
from sklearn.metrics import jaccard_similarity_score

df_moods = my_database.moods.str.join(',').str.get_dummies(sep=',')
df_genres_moods = pd.concat((my_database.genres, df_moods), axis=1)

In [ ]:
def recommend_similar_songs(audio_features, lyrics_features):
    
    genre = predict_genres.predict(audio_features.reshape((1, -1)))
    
    if lyrics_features is not None:
        
        lyrics_features_clean = clean_text(lyrics_features)
    
        moods_1 = predict_moods_audio.predict_proba(audio_features.reshape((1, -1)))
        moods_2 = predict_moods_lyrics.predict_proba([lyrics_features_clean])
        moods = (moods_1*2 + moods_2*1)*1/3
    
    else:
        moods = predict_moods_audio.predict_proba(audio_features.reshape((1, -1)))
    
    predict_result = np.concatenate((genre, moods))
    
    similarity = df_genres_moods.apply(lambda x: jaccard_similarity_score(predict_result, x.tolist()), axis=1)
    similarity = similarity.sort_values(ascending=False)[0:50]
    
    top_10 = my_database.iloc[similarity.sample(10).index]
    
    result = [ Song(artist=row["artist"], title=row['name']) for idx, row in top_10.iterrows()]
    final_result_dictionary = dict(playlist=result)
    
    return final_result_dictionary

In [63]:
predict_genres_moods = recommend_similar_songs(audio_features, lyrics_features)
print(predict_genres_moods)

/home/dks/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


{'playlist': [Song(artist='Madeon', title='The City'), Song(artist='Zac Brown Band', title="Whiskey's Gone"), Song(artist='Tiësto', title='United (Ultra Music Festival Anthem)')]}
